In [33]:
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from transformer import TransformerModel


In [35]:
# initalize model
model = TransformerModel(emb_sz=100, num_heads=5, key_dim=64, vocab_size=15000)

# compile model with Adam optimizer and masked loss and masked accuracy 
model.compile(optimizer='Adam', loss=TransformerModel.masked_loss, metrics=[TransformerModel.masked_accuracy])

# load weights
model.load_weights('../models/weights/2blocks-fixedlr-datav2_75epochs_weights')

In [36]:
# get train content data from pickle 
with open('../data/embeddings/train_content_embeddings.pkl','rb') as f:
    train_content_emb = pickle.load(f)

# get train title data from pickle 
with open('../data/embeddings/train_title_embeddings.pkl','rb') as f:
    train_title_emb = pickle.load(f)

# get train title labels data from pickle 
with open('../data/embeddings/train_title_labels.pkl','rb') as f:
    train_title_tokens = pickle.load(f)

# get test content data from pickle 
with open('../data/embeddings/test_content_embeddings.pkl','rb') as f:
    test_content_emb = pickle.load(f)
    
# get test title data from pickle 
with open('../data/embeddings/test_title_embeddings.pkl','rb') as f:
    test_title_emb = pickle.load(f)
    
# get test title labels data from pickle 
with open('../data/embeddings/test_title_labels.pkl','rb') as f:
    test_title_tokens = pickle.load(f)

# get word dictionary data from pickle 
with open('../data/embeddings/title_index_word.pkl','rb') as f:
    word_dict = pickle.load(f)


In [37]:
# function to convert indexes to words using defined dict 
def sentence_from_ind(indexes):
    sentence = ""
    
    for index in indexes:
   
        sentence += word_dict[index]
        sentence += " "
    
    return sentence 

In [38]:
# create datasets for training and testing 
train_content_data = tf.convert_to_tensor(train_content_emb)
train_title_data = tf.convert_to_tensor(train_title_emb)
train_title_labels = tf.convert_to_tensor(train_title_tokens)

test_content_data = tf.convert_to_tensor(test_content_emb)
test_title_data = tf.convert_to_tensor(test_title_emb)
test_title_labels = tf.convert_to_tensor(test_title_tokens)

In [39]:
# create predictions 
pred_train = model.predict((train_content_data[:500], train_title_data[:500]))
pred_test = model.predict((test_content_data[:500], test_title_data[:500]))

16/16 [==============================] - 2s 146ms/step


In [40]:
# save training results to csv
predicted_sent_list = []
true_sent_list = []
BLEU_scores = []

for i in range(0, 500):
    tokens = np.argmax(pred_train[i],axis=1)
    true = train_title_labels[i].numpy().reshape((16,))

    pred_sent = sentence_from_ind(tokens)
    true_sent = sentence_from_ind(true)

    predicted_sent_list.append(pred_sent)
    true_sent_list.append(true_sent)
    BLEU_scores.append(sentence_bleu([true_sent.split()],pred_sent.split(), weights=(1, 0, 0, 0)))

df_train = pd.DataFrame({'predicted':predicted_sent_list, 'true':true_sent_list, 'BLEU':BLEU_scores})
df_train = df_train.sort_values(by='BLEU', ascending=False)
# df_train.to_csv('../results/train_results.csv')
df_train

/Users/lihengpanza/miniforge3/envs/csci2470-project/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/lihengpanza/miniforge3/envs/csci2470-project/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/lihengpanza/miniforge3/envs/csci2470-project/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many

,predicted,true,BLEU
139,oliver blocked chinas weibo website clamps dow...,john oliver blocked chinas weibo website clamp...,0.8125
184,lessons american who designed the modern sneak...,meet the american who designed the modern snea...,0.8125
322,ag ag ripped for gop challenger claim frivolou...,democrat minnesota ag ripped by gop challenger...,0.7500
261,paul staffer suffered deep knife wound to head...,rand paul staffer suffered deep knife wound to...,0.7500
385,weingartens is wsj retort on schools schools e...,randi weingartens [UNK] wsj retort on schools ...,0.7500
...,...,...,...
209,is government doctors the is government your i...,what are [UNK] rights,0.0000
47,agency education for heads decision bill im to...,a public health victory,0.0000
2,important the record a houses team we learning...,education,0.0000
181,education paris american of russias a small a ...,[UNK] meets the world,0.0000


In [41]:
# save test predictions to csv
predicted_sent_list = []
true_sent_list = []
BLEU_scores = []

for i in range(0,500):
    tokens = np.argmax(pred_test[i],axis=1)
    true = test_title_labels[i].numpy().reshape((16,))

    pred_sent = sentence_from_ind(tokens)
    true_sent = sentence_from_ind(true)

    predicted_sent_list.append(pred_sent)
    true_sent_list.append(true_sent)
    BLEU_scores.append(sentence_bleu([true_sent.split()],pred_sent.split(), weights=(1, 0, 0, 0)))

df_test = pd.DataFrame({'predicted':predicted_sent_list, 'true':true_sent_list, 'BLEU':BLEU_scores})
df_test = df_test.sort_values(by='BLEU', ascending=False)
# df_test.to_csv('../results/test_results.csv')
df_test

/Users/lihengpanza/miniforge3/envs/csci2470-project/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/lihengpanza/miniforge3/envs/csci2470-project/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/lihengpanza/miniforge3/envs/csci2470-project/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many

,predicted,true,BLEU
314,scott ceo elon ceo his his on social praise on...,rumble ceo tells tucker carlson his company is...,0.3750
351,19 to to covid 5 5 for thursday covid before v...,de blasio to pay kids 100 to get the covid 19 ...,0.3125
485,south covid covid johnson reported reported jo...,johnson johnson covid vaccine linked to death ...,0.3125
377,mask pan to mask masks mask for public transit...,new york continues to enforce mask mandate for...,0.3125
444,department to to aid to on 31 31 drone by atta...,us state department proposes [UNK] sale of [UN...,0.3125
...,...,...,...
402,press controversial globe progress sparks on e...,brazil [UNK] all its votes in hours it still f...,0.0000
334,claims breaking ukraines russian ukraine of co...,russias claim to seize a ukrainian town expose...,0.0000
335,protests lefts dire join conference to regimes...,cop26 protesters in glasgow push for action fr...,0.0000
399,kudlow greta should climate dies kerry based h...,abby [UNK] challenges jane fonda on [UNK] gree...,0.0000


In [42]:
df_test['BLEU'].mean()

0.09025